In [1]:
import json
import numpy as np
import openai
# remember to set your OpenAI api key
openai.api_key = ''

In [2]:
def get_gpt_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return np.array(openai.Embedding.create(input = [text], model=model)['data'][0]['embedding'])

In [ ]:
with open('input_data/GenePT_emebdding_v2/NCBI_summary_of_genes.json', 'r') as f:
    summary_all_genes = json.load(f)

len(summary_all_genes.keys())

In [4]:
from glob import glob
import pandas as pd 

slide_outs = '/ix/djishnu/alw399/SLIDE_SWING/slide-outputs/totalCD8_TEDDY_yKIR/CTL/0.1_1_out'
sig_lfs = glob(f'{slide_outs}/feature_list*')

genes = []
for path in sig_lfs:
    genes.extend(list(pd.read_csv(path, sep='\t')['names']))

len(genes)


56

In [5]:
NCBI_cleaned_summary_of_genes = {k: summary_all_genes[k] for k in genes if k in summary_all_genes}
len(NCBI_cleaned_summary_of_genes.keys())

43

In [6]:
set(genes) - set(NCBI_cleaned_summary_of_genes.keys())

{'AC006369.1',
 'AC009133.3',
 'AC093865.1',
 'ADPRHL2',
 'AF117829.1',
 'AL353194.1'}

In [7]:
# with open(f"{data_dir}NCBI_cleaned_summary_of_genes.json", 'r') as file:
#     NCBI_cleaned_summary_of_genes = json.load(file)

In [8]:
gpt_gene_name_to_embedding_clean_text = {}

In [ ]:
import pickle 

with open('/ix/djishnu/alw399/SLIDE_PLM/notebooks/GenePT-main/input_data/GenePT_emebdding_v2/GenePT_gene_embedding_ada_text.pickle', 'rb') as file:
    gpt_gene_name_to_embedding = pickle.load(file)

In [12]:
len(gpt_gene_name_to_embedding.keys())

93800

In [14]:
gpt_gene_name_to_embedding_clean_text = {k: v for k, v in gpt_gene_name_to_embedding.items() if k in NCBI_cleaned_summary_of_genes.keys()}
len(gpt_gene_name_to_embedding_clean_text.keys())

43

In [17]:
np.array(gpt_gene_name_to_embedding_clean_text['ADAM8']).shape

(1536,)

In [9]:
GPT_DIM = 1536 # fix GPT embeddings
for key, text in sorted(NCBI_cleaned_summary_of_genes.items()):
    if key not in gpt_gene_name_to_embedding_clean_text:
        print('key',key)
        if NCBI_cleaned_summary_of_genes[key] == '': 
            # if the dictionary does not have information about a gene
            NCBI_cleaned_summary_of_genes[key] = np.zeros(GPT_DIM) # it's hard coded to be 0
        else:
            NCBI_cleaned_summary_of_genes[key] = get_gpt_embedding(text) 

key ADAM8


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
with open(f"{data_dir}GPT_3_5_gene_embeddings.pickle", "wb") as fp:
    pickle.dump(gpt_gene_name_to_embedding_clean_text, fp)

#### We provide an example to get GTP to complete content for gene ALPP. For your use case it could be any other genes of interest.

In [18]:
gene_name_to_GPT_response = {}

In [ ]:
gene_completion_test = ['ALPP']
for gene in sorted(gene_completion_test):
    if gene not in gene_name_to_GPT_response:
        print('gene name',gene)
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                    messages=[{"role": "user", "content": f"Tell me about gene {gene}"}])
        gene_name_to_GPT_response[gene] = completion.choices[0].message.content

In [ ]:
def get_gene_summary(gene, gene_name_to_GPT_response):
    if gene not in gene_name_to_GPT_response:
        print('gene name',gene)
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                    messages=[{"role": "user", "content": f"Tell me about gene {gene}"}])
        gene_name_to_GPT_response[gene] = completion.choices[0].message.content
    
    return gene_name_to_GPT_response[gene]

get_gene_summary('ADAM8', gene_name_to_GPT_response)

Let's also insepct the results to make sure that GPT-3.5 was not hallucinating here --- indeed the output from this API call looks biologically relevant:

In [ ]:
gene_name_to_GPT_response['ALPP']

Output from GPT: `'Gene ALPP, also known as Alkaline Phosphatase, Placental-Like (ALPPL2), is a gene that encodes an enzyme called Alkaline phosphatase placental-like. This gene is located on human chromosome 2q37.3.\n\nAlkaline phosphatase is an enzyme that removes phosphate groups from various molecules, and it is involved in many important physiological processes. ALPP is primarily expressed in the placenta, but it can also be found in other tissues like liver, kidney, and intestine.\n\nThe ALPP gene is thought to play a role in placental development and function, as well as in fetal and postnatal bone metabolism. Studies have shown that mutations or dysregulation of ALPP expression can lead to various disorders and diseases. For example, decreased ALPP activity has been associated with hypophosphatasia, a rare genetic disorder characterized by defective bone mineralization.\n\nIn addition, ALPP expression has been linked to certain types of cancer. Increased ALPP activity has been observed in some malignancies, such as colorectal cancer and hepatocellular carcinoma, and it has been suggested as a potential marker for these cancers.\n\nOverall, the ALPP gene and its encoded enzyme, Alkaline phosphatase placental-like, have important roles in placental development, bone metabolism, and potentially in certain cancers. Further research is still needed to fully understand the exact mechanisms and implications of this gene in various physiological processes and diseases.'`